# Inoculation Protocol

## Method

## Layout
![Inoculation Layout](Images/Inoculation_layout.PNG)

## Worklist Generation

We developed an internal-use package to automate worklist generation in a consistent and reliable way for all kinds of experimental designs. The package is called `worklistgen`. (This is a work in progress, as we continue adding new features and fixing bugs. If you have any suggestions, please let us know.)

### Usage Example

In [ ]:
# import worklistgen and other necessary libraries
import pandas as pd
from worklistgen.core import Source, Destination, Worklist

import random
from itertools import combinations

We are going to have two sources. One for the species we are going to use in our experiemnt and the other one for the different media conditions that we are going to use.

In [ ]:
# Define Species
species = ['EF', 'FP', 'BO', 'CD', 'DI', 'EL', 
           'BU', 'AC', 'GH', 'IJ', 'KL', 'MN', 
           'OP', 'QR', 'ST', 'UV', 'WX', 'YZ']
# Define Media
media = ['DM38', 'LB', 'M9', 'M9Glucose', 
         'M9Glycerol', 'M9Acetate', 'M9Propionate', 'M9Butyrate']

For our worklist, we will need a table containing all the different combinations of species and media conditions. Here we are going to generate one randomly, but you can also use your own table. The only requirement is that the first column contains the species separated by "-" (`community`) and the second column contains the media conditions separated by "-" (`condition`). The rest of the columns will be ignored.

In [7]:
# Randomly generate communities
community_combinations = []
for size in range(1, len(species) + 1):
    community_combinations.extend(combinations(species, size))
random_communities = random.sample(community_combinations, 60)
communities = ['-'.join(community) for community in random_communities]

# Randomly generate conditions
condition_combinations = []
for size in range(1, len(media) + 1):
    condition_combinations.extend(combinations(media, size))
random_conditions = random.sample(condition_combinations, 60)
conditions = ['-'.join(condition) for condition in random_conditions]

# Build the experiments DataFrame
experiments = pd.DataFrame({
    'community': communities,
    'condition': conditions,
})
experiments.head(10)

,community,condition
0,EF-BO-DI-EL-GH-IJ-KL-UV-WX-YZ,LB-M9-M9Acetate-M9Butyrate
1,BO-CD-DI-BU-GH-IJ-MN-OP-UV-WX,LB-M9-M9Glucose-M9Glycerol-M9Acetate-M9Butyrate
2,EF-BO-EL-BU-IJ-MN-YZ,DM38-M9-M9Glycerol-M9Acetate-M9Butyrate
3,EF-FP-CD-DI-EL-BU-GH-IJ-MN-OP-QR,LB-M9-M9Glucose-M9Glycerol-M9Acetate-M9Propion...
4,EF-DI-AC-IJ-QR-ST-WX,LB-M9Glucose-M9Glycerol-M9Propionate
5,BO-CD-BU-AC-GH-KL-OP-ST,LB-M9Glucose-M9Glycerol-M9Propionate-M9Butyrate
6,EF-FP-EL-KL-MN-QR-ST,DM38-LB-M9Glycerol-M9Propionate-M9Butyrate
7,FP-BO-AC-GH-IJ-MN-OP-YZ,M9-M9Glucose-M9Glycerol-M9Propionate-M9Butyrate
8,EF-FP-DI-EL-AC-KL-QR-ST-WX,DM38-M9Glucose-M9Propionate
9,EF-DI-EL-GH-MN-OP-QR-WX,LB-M9Acetate-M9Propionate-M9Butyrate


Now, we are going to create our worklist. For this we need define:
* Our sources -> using the `Source` class
* Our destination -> using the `Destination` class
    - Here we will also define the number of replicates we want for each combination of species and media condition, the blanks we want to add and the number of replicates we want for the blanks.

In [9]:
# Define sources
source1 = Source(name='species_source', source_type='species', labware_type='96wellplate', input=species)
source2 = Source(name='media_source', source_type='media', labware_type='tuberack', input=media)

# Define destination
destination = Destination(name='destination', labware_type='96wellplate', experiments=experiments, blanks=['M9Propionate', 'M9Butyrate'], replicates=3, blanks_replicates=2)

# Create worklist
worklist = Worklist(name='worklist', total_volume=100)
worklist.add_source(source1)
worklist.add_source(source2)
worklist.add_destination(destination)

# Generate worklist
worklist.generate_worklist()
worklist.worklist_df

,Source_plate_Name,Source_Plate_Type,Source_Plate_Well,Destination_plate_Name,Destination_Plate_Type,Destination_Plate_Well,Transfer_Volume
0,media_source-1,tuberack,A7,destination-1,96wellplate,A1,100.0
1,media_source-1,tuberack,A7,destination-1,96wellplate,B1,100.0
2,media_source-1,tuberack,A8,destination-1,96wellplate,C1,100.0
3,media_source-1,tuberack,A8,destination-1,96wellplate,D1,100.0
4,species_source-1,96wellplate,A1,destination-1,96wellplate,E1,10.0
...,...,...,...,...,...,...,...
2282,media_source-1,tuberack,A2,destination-2,96wellplate,H11,20.0
2283,media_source-1,tuberack,A3,destination-2,96wellplate,H11,20.0
2284,media_source-1,tuberack,A6,destination-2,96wellplate,H11,20.0
2285,media_source-1,tuberack,A7,destination-2,96wellplate,H11,20.0
